In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from keybert import KeyBERT
import time
from tqdm.notebook import tqdm

In [2]:
sds_text = pd.read_csv('../data/sds_text.csv')

<IPython.core.display.Javascript object>

In [3]:
sds_text

,Unnamed: 0,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_day,host_episode,speaker,episode_split_text
0,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,This is episode number one with ex-chemical e...
1,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,"Hey guys, welcome to the Podcast. I’ve got Ru..."
2,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,Thank you! Thanks for having me over. I’m doi...
3,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,Awesome. It’s great to hear you and for those...
4,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,"Yeah sure. So, I’m the senior manager of anal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46767,681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Jon Krohn,"Yeah, right. Yeah, as I mean, it actually, it ..."
46768,681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Matar Haller,Happy to.
46769,681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Jon Krohn,"Nice. Well, yeah, so you mentioned potentially..."
46770,681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Matar Haller,Thank you for having me. This was fascinating ...


In [4]:
sds_processed = pd.DataFrame(sds_text.groupby('episode_number')['episode_split_text'].agg(lambda x: ' '.join(x))).reset_index()

<IPython.core.display.Javascript object>

In [5]:
sds_processed['episode_keywords'] = None

In [6]:
sds_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   episode_number      680 non-null    int64 
 1   episode_split_text  680 non-null    object
 2   episode_keywords    0 non-null      object
dtypes: int64(1), object(2)
memory usage: 16.1+ KB


In [7]:
sds_processed

,episode_number,episode_split_text,episode_keywords
0,1,This is episode number one with ex-chemical e...,None
1,2,This is session number two with machine learn...,None
2,3,This is episode number three with Nanophysics ...,None
3,4,This is episode four with business strategy e...,None
4,5,This is episode number five with forensics inv...,None
...,...,...,...
675,679,This is episode number 679 with George Matthew...,None
676,680,This is episode number 680 with Allegra Alessi...,None
677,681,"This is episode number 681 with Matt Harrison,...",None
678,682,"This is episode number 682 with Mico Yuk, host...",None


In [8]:
sds_processed['episode_split_text'].loc[2]

"This is episode number three with Nanophysics PHD turned data scientists Wilson Pok.Welcome to the Super Data Science Podcast. My name is Kirill Eremenko, Data Science Coach and lifestyle entrepreneur. Each week we bring you inspiring people and ideas to help you build your successful career in Data Science. Thanks for being here today and now let's make the complex simple.Welcome everybody to the Super Data Science podcast. Super pumped to have you here on board today. This podcast is all about interviewing the top data scientists in the world to help you build your successful career in data science.Today our guest is Wilson Pok. Wilson is a friend of mine who I met back at Deloitte when I just started. Only worked for a few weeks together but Wilson's such an approachable guy. He’s one of those people that are so open, so outgoing and at the same time very modest that we got along very well and stayed in touch.Now he's in Sydney. He's working at a consulting firm called Ambiata. Bef

In [9]:
def keyword_extractor(doc):
    kw_model = KeyBERT()
    global sds_processed
    stopwords = list(STOP_WORDS)
    
    for i, episode_text in doc.iteritems():
        keywords = kw_model.extract_keywords(episode_text, keyphrase_ngram_range=(1, 1), stop_words = stopwords, top_n = 10)
        sds_processed.at[i, 'episode_keywords'] = keywords

In [10]:
keyword_extractor(sds_processed['episode_split_text'][:10])

/Applications/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
/Applications/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
/Applications/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
/Applications/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop word

In [11]:
sds_processed.head(10)

,episode_number,episode_split_text,episode_keywords
0,1,This is episode number one with ex-chemical e...,"[(podcast, 0.4874), (episode, 0.4251), (superd..."
1,2,This is session number two with machine learn...,"[(hadelin, 0.4046), (podcast, 0.3886), (expert..."
2,3,This is episode number three with Nanophysics ...,"[(podcast, 0.4022), (wilson, 0.3662), (superda..."
3,4,This is episode four with business strategy e...,"[(brendan, 0.4234), (superdatascience, 0.4127)..."
4,5,This is episode number five with forensics inv...,"[(dmitry, 0.5054), (podcast, 0.4389), (episode..."
5,6,"This is episode number 6, with expert financia...","[(podcast, 0.3899), (superdatascience, 0.3891)..."
6,7,"This is episode number 7, with top analytics c...","[(podcast, 0.463), (superdatascience, 0.3851),..."
7,8,"This is episode number 8, with Finance Directo...","[(ubisoft, 0.5044), (ulf, 0.3946), (enterprise..."
8,9,"This is episode number 9, with neuroscience Ph...","[(superdatascience, 0.4645), (academia, 0.4433..."
9,10,"This is episode number 10, with senior data sc...","[(superdatascience, 0.3655), (episode, 0.3492)..."


In [17]:
sds_processed['episode_keywords'].loc[9]

[('superdatascience', 0.3655),
 ('episode', 0.3492),
 ('podcast', 0.3393),
 ('yaw', 0.2831),
 ('kirill', 0.272),
 ('expertise', 0.2716),
 ('mentor', 0.2397),
 ('interviews', 0.2393),
 ('lecturer', 0.2385),
 ('super', 0.2358)]

In [ ]:
def keyword_extractor(doc):
    kw_model = KeyBERT()
    keyword = ''
    global sds_processed
    stopwords = list(STOP_WORDS)
    #global stopwords 
    
    for keyword in doc:
        keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1, 1), stop_words = stopwords)
        sds_processed['episode_keywords'] = sds_processed['episode_keywords'].replace(None, keywords)
    

In [ ]:
def keyword_extractor(doc):
    kw_model = KeyBERT()
    global sds_processed
    stopwords = list(STOP_WORDS)
    
    keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=stopwords)
    sds_processed['episode_keywords'] = sds_processed['episode_keywords'].apply(lambda x: keywords if x == '' else x)


In [ ]:
keyword_extractor(sds_processed['episode_split_text'])

In [ ]:
sds_processed

In [ ]:
keyword_extractor(sds_processed['episode_split_text'][:5])

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_split_text'][1:2].apply(keyword_extractor)

In [ ]:
sds_processed

In [ ]:
stopwords = list(STOP_WORDS)

In [ ]:
def keyword_extractor(doc):
    kw_model = KeyBERT()
    global stopwords 
    keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words = stopwords)
    return keywords

In [ ]:
#sds_processed = pd.DataFrame(sds_text.groupby('episode_number')['episode_split_text'].agg(lambda x: ' '.join(x)))

In [ ]:
sds_1 = grouped_df.loc[0]

In [ ]:
sds_1.info()

In [ ]:
sds_1 = grouped_df[grouped_df['episode_number'] == 1]

In [ ]:
sds_1

In [ ]:
grouped_df

In [ ]:
grouped_df = sds_text.groupby('episode_number')['episode_split_text'].apply(' '.join).reset_index()
sds_1['keywords'] = sds_1['episode_split_text'].apply(keyword_extractor)

In [ ]:
sds_1

In [ ]:
keyword_extractor(sds_processed['episode_split_text'][1])

In [ ]:
stopwords

In [ ]:
sds_processed['episode_split_text'][1:5].apply(keyword_extractor)

In [ ]:
[600:]